In [1]:
import os
from types import SimpleNamespace
import tensorflow as tf
from tensorflow.python import debug as tf_debug

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Data Preparation

Convert data to TFRecord binary format

In [2]:
def _data_path(data_directory: str, name: str) -> str:
    """Construct a full path to a TFRecord file to be stored in the 
    data_directory. Will also ensure the data directory exists

    Args:
        data_directory: The directory where the records will be stored
        name:           The name of the TFRecord

    Returns:
        The full path to the TFRecord file
    """
    if not os.path.isdir(data_directory):
        os.makedirs(data_directory)

    return os.path.join(data_directory, '{}.tfrecords'.format(name))


def _int64_feature(value: int) -> tf.train.Features.FeatureEntry:
    """Create a Int64List Feature

    Args:
        value: The value to store in the feature

    Returns:
        The FeatureEntry
    """
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _bytes_feature(value: str) -> tf.train.Features.FeatureEntry:
    """Create a BytesList Feature

    Args:
        value: The value to store in the feature

    Returns:
        The FeatureEntry
    """
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def convert_to(data_set, name: str, data_directory: str, num_shards: int=1):
    """Convert the dataset into TFRecords on disk

    Args:
        data_set:       The MNIST data set to convert
        name:           The name of the data set
        data_directory: The directory where records will be stored
        num_shards:     The number of files on disk to separate records into
    """

    num_examples, rows, cols, depth = data_set.images.shape

    data_set = list(zip(data_set.images, data_set.labels))

    def _process_examples(example_dataset, filename: str):
        print('Processing {} data'.format(filename))
        dataset_length = len(example_dataset)
        with tf.python_io.TFRecordWriter(filename) as writer:
            for index, (image, label) in enumerate(example_dataset):
                sys.stdout.write('\rProcessing sample {} of {}'.format(
                    index + 1, dataset_length))
                sys.stdout.flush()

                image_raw = image.tostring()
                example = tf.train.Example(features=tf.train.Features(feature={
                    'height': _int64_feature(rows),
                    'width': _int64_feature(cols),
                    'depth': _int64_feature(depth),
                    'label': _int64_feature(int(label)),
                    'image_raw': _bytes_feature(image_raw)
                }))
                writer.write(example.SerializeToString())
            print()

    if num_shards == 1:
        _process_examples(data_set, _data_path(data_directory, name))
    else:
        sharded_dataset = np.array_split(data_set, num_shards)
        for shard, dataset in enumerate(sharded_dataset):
            _process_examples(dataset, _data_path(
                data_directory, '{}-{}'.format(name, shard + 1)))


def convert_to_tf_record(args):
    """Convert the TF MNIST Dataset to TFRecord formats

    Args:
        data_directory: The directory where the TFRecord files should be stored
    """

    mnist = input_data.read_data_sets(
        "/tmp/tensorflow/mnist/input_data",
        reshape=False
    )

    convert_to(mnist.validation, 'validation', args.data_directory)
    convert_to(mnist.train, 'train', args.data_directory, num_shards=10)
    convert_to(mnist.test, 'test', args.data_directory)

In [ ]:
prep_args = SimpleNamespace(
    data_directory=os.path.expanduser('~/data/mnist')
)

convert_to_tf_records(prep_args)

## Data input

Setup our data `input_fn`

In [ ]:
def get_feature_columns(with_label=True):
    features = {
        'image_raw': tf.FixedLenFeature([], tf.string)
    }
    if with_label:
        features['label'] = tf.FixedLenFeature([], tf.int64)
    return features


def make_input_fn(filenames, batch_size=1024, shuffle=False):

    def _parser(record):
        image = tf.decode_raw(record['image_raw'], tf.float32)

        label = tf.cast(record['label'], tf.int32)

        return {
            'image': image
        }, label

    def _input_fn():
        dataset = tf.contrib.data.make_batched_features_dataset(
            file_pattern=filenames, batch_size=batch_size, 
            features=get_feature_columns(),
            shuffle=shuffle, sloppy_ordering=True
        )

        dataset = dataset.map(_parser, num_parallel_calls=os.cpu_count())
        return dataset

    return _input_fn

## Model definition

Define our CNN model 

In [ ]:
def model_fn(features, labels, mode, params):
    """CNN architecture to process 28x28x1 MNIST images
    Arguments:
        features: tensor of MNIST images
        mode: estimator mode
        params: dictionary of hyperparameters

    Returns:
        Tensor of the final layer output without activation
    """

    is_training = mode == tf.estimator.ModeKeys.TRAIN

    with tf.name_scope('Input'):
        # Input Layer
        input_layer = tf.reshape(
            features['image'], [-1, 28, 28, 1], name='input_reshape')
        tf.summary.image('input', input_layer)

    with tf.name_scope('Conv_1'):
        # Convolutional Layer #1
        conv1 = tf.layers.conv2d(
            inputs=input_layer,
            filters=32,
            kernel_size=(5, 5),
            padding='same',
            activation=tf.nn.relu,
            trainable=is_training)

        # Pooling Layer #1
        pool1 = tf.layers.max_pooling2d(
            inputs=conv1, pool_size=(2, 2), strides=2, padding='same')

    with tf.name_scope('Conv_2'):
        # Convolutional Layer #2 and Pooling Layer #2
        conv2 = tf.layers.conv2d(
            inputs=pool1,
            filters=64,
            kernel_size=(5, 5),
            padding='same',
            activation=tf.nn.relu,
            trainable=is_training)

        pool2 = tf.layers.max_pooling2d(
            inputs=conv2, pool_size=(2, 2), strides=2, padding='same')

    with tf.name_scope('Dense_Dropout'):
        # Dense Layer
        pool2_flat = tf.layers.flatten(pool2)
        dense = tf.layers.dense(
            inputs=pool2_flat, units=1024, activation=tf.nn.relu, trainable=is_training)
        dropout = tf.layers.dropout(
            inputs=dense, rate=params['dropout_rate'], training=is_training)

    with tf.name_scope('Predictions'):
        # Logits Layer
        logits = tf.layers.dense(
            inputs=dropout, units=10, trainable=is_training)

    head = tf.contrib.estimator.multi_class_head(
        n_classes=10)

    optimizer = tf.train.AdamOptimizer(learning_rate=params['learning_rate'])
    return head.create_estimator_spec(
        features, mode, logits, labels, optimizer=optimizer,
    )

## Training

In [ ]:
def get_train_spec(args, hooks):
    train_files = os.path.join(args.data_directory, 'train-*.tfrecords')
    
    train_input_fn = make_input_fn(
        train_files, batch_size=args.batch_size)
    
    train_spec = tf.estimator.TrainSpec(
        input_fn=train_input_fn, max_steps=args.max_steps, hooks=hooks)
    
    return train_spec

In [ ]:
def get_eval_spec(args):
    eval_files = os.path.join(args.data_directory, 'validation.tfrecords')
    
    eval_input_fn = make_input_fn(eval_files, batch_size=1)
    
    eval_spec = tf.estimator.EvalSpec(eval_input_fn)
    
    return eval_spec

In [ ]:
def train(args):
    """Run training and evaluation
    """

    run_config = tf.estimator.RunConfig()

    hparams = {
        'learning_rate': args.learning_rate,
        'dropout_rate': 0.4
    }

    mnist_classifier = tf.estimator.Estimator(
        model_fn=model_fn,
        model_dir=args.model_directory,
        config=run_config,
        params=hparams
    )

    hooks = []

    if args.debug_port is not None:
        debug_hook = tf_debug.TensorBoardDebugHook(
            "localhost:{}".format(args.debug_port))
        hooks.append(debug_hook)

    tf.estimator.train_and_evaluate(
        mnist_classifier, get_train_spec(args, hooks), get_eval_spec(args))
    
    return mnist_classifier

In [ ]:
args = SimpleNamespace(
    model_directory='',
    data_directory='',
    learning_rate=0.4,
    batch_size=1024,
    max_steps=400,
    debug_port=None
)

classifier = train(args)

In [ ]:
def serving_input_receiver_fn():
    
    feature_spec = {
        'image': tf.FixedLenFeature([], dtype=tf.string)
    }
    
    default_batch_size = None # the number of query examples expected per batch. Leave unset for variable batch size (recommended).
    
    serialized_tf_example = tf.placeholder(
        dtype=tf.string, shape=[default_batch_size], 
        name='input_image_tensor')
    
    received_tensors = { 'images': serialized_tf_example }
    features = tf.parse_example(serialized_tf_example, feature_spec)
    
    def map_input(image_raw):
        image_decoded = tf.image.decode_jpeg(image_raw, channels=1)
        # Convert from full range of uint8 to range [0,1] of float32.
        image_decoded_as_float = tf.image.convert_image_dtype(image_decoded, dtype=tf.float32)
        # Resize to expected
        image_resized = tf.image.resize_images(image_decoded_as_float, size=(28, 28))
        
        return image_resized
    
    features['image'] = tf.map_fn(map_input, features['image'], dtype=tf.float32)
    
    return tf.estimator.export.ServingInputReceiver(features, received_tensors)

classifier.export_savedmodel('mnist-exports', serving_input_receiver_fn)